In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import pydicom
import h5py

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset
from matplotlib.pyplot import imread
import glob
import os
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import torch.nn as nn
from torchvision.models import alexnet, vgg16, resnet152, resnet18, vgg19

%matplotlib inline

import pyro
from pyro.distributions import Normal, Categorical
from pyro.infer import SVI, Trace_ELBO, TraceMeanField_ELBO, TraceGraph_ELBO
from pyro.optim import Adam
from pyro import poutine
import pyro.optim as pyroopt
import pyro.distributions as dist
import pyro.contrib.bnn as bnn
import matplotlib.pyplot as plt
import seaborn as sns
from torch.distributions.utils import lazy_property
import math

import torch.nn.functional as nnf
from torch.utils.data import random_split
from torch.optim import SGD 
from torch.distributions import constraints
import torchvision as torchv
import torchvision.transforms as torchvt
import pickle

In [8]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Function for moving tensor or model to GPU
def cuda(xs):
    if torch.cuda.is_available():
        if not isinstance(xs, (list, tuple)):
            return xs.cuda()
        else:
            return [x.cuda() for x in xs]
    else:
        return xs

# Custom class for defining dataset for training with augmentation
class Dataset_Hdf5(Dataset):

    def __init__(self, path, data_type, cv_index):
        """ Intialize the dataset
        """
        self.path = path
        self.file = h5py.File(path, 'r')
        self.images = self.file['data'][self.file['cv_indices_{}_{}'.format(data_type, cv_index)]]
        self.labels = self.file['labels'][self.file['cv_indices_{}_{}'.format(data_type, cv_index)]]
                
        self.len = self.images.shape[0]
        if data_type == 'train':
          self.transform = transforms.Compose([
                                              transforms.ToPILImage(),
                                              transforms.RandomRotation((0, 360)),
                                              transforms.RandomHorizontalFlip(),
                                              transforms.RandomVerticalFlip(),
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.5], [0.5])])
        else:
          self.transform = transforms.Compose([transforms.ToTensor(),
                                              transforms.Normalize([0.5], [0.5])
                                              ])

    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        # unsqueeze adds dimension to image -> converts to 1x224x224 since we don't have rgb
        return cuda(self.transform(self.images[index].astype('float32'))), \
                cuda(torch.tensor(self.labels[index], dtype=torch.long))

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

# 64 * [224 * 224], 0

In [0]:
# Need to define loaders for train and test in PyTorch
ddms_train_loader = torch.utils.data.DataLoader(Dataset_Hdf5('/content/drive/My Drive/ddsm_cropped_all.hdf5', 'train', 3), 
                                                batch_size=16, shuffle=True)
ddms_test_loader = torch.utils.data.DataLoader(Dataset_Hdf5('/content/drive/My Drive/ddsm_cropped_all.hdf5', 'test', 3), 
                                               batch_size=16, shuffle=True)

In [0]:
# Try transfer learning
res_net = resnet18(pretrained=True)
# First we freeze all layers
for param in res_net.parameters():
    param.requires_grad = False # frozen layer


# Now we make later layers trainable
for param in res_net.layer4.parameters():
    param.requires_grad = True
for param in res_net.layer3.parameters():
    param.requires_grad = True
for param in res_net.avgpool.parameters():
    param.requires_grad = True
    
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = res_net.fc.in_features
# Replace 1 FC with 3 new FC
res_net.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=2, bias=True))
 
# Move ResNet to GPU
res_net.cuda()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:00<00:00, 52126432.87it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [1]:
# Adam optimizer
# Mostly following learning rates from paper
ddms_optimizer_res_net = torch.optim.Adam(
    [                                         {"params": res_net.layer3.parameters(), "lr": 0.0001},
                                              {"params": res_net.layer4.parameters(), "lr": 0.0001},
                                              {"params": res_net.avgpool.parameters(), "lr": 0.0001},    
                                              {"params": res_net.fc[0].parameters(), "lr": 0.0001},
                                              {"params": res_net.fc[2].parameters(), "lr": 0.001},
                                              {"params": res_net.fc[4].parameters(), "lr": 0.002},

                                           ],  
                                lr=0.0001, betas=(0.5, 0.999))

In [0]:
# Training function for deterministic network
def train(net, train_loader, criterion, optimizer, net_name, num_epochs=25):
    net.train()
    train_acc_max = 0
    test_acc_max = 0

    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        net.train()

        total = 0
        correct = 0
   
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.expand(-1, 3, -1, -1))
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        
        scheduler.step()
        
        print('End of epoch {}, Loss {}'.format(epoch + 1, running_loss / len(train_loader)))
        
        train_acc = correct / total
        test_acc = test(net, ddms_test_loader)
        
        # Saving best checkpoint based on performance on test data
        if train_acc > train_acc_max:
            train_acc_max = train_acc
            save_checkpoint(epoch + 1, net, optimizer, train_acc, test_acc, net_name, 'train')
            
        if test_acc > test_acc_max:
            test_acc_max = test_acc
            save_checkpoint(epoch + 1, net, optimizer, train_acc, test_acc, net_name, 'test')

    print('Finished Training')
    
def test(net, test_loader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            outputs = net(images.expand(-1, 3, -1, -1))
            _, predicted = torch.max(outputs, 1)
#             predicted = predicted.float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = correct / total
    print('Accuracy of the network on the images: %d %%' % (100 * acc))
    return acc
    
def save_checkpoint(epoch, net, optimizer, train_acc, test_acc, net_name, param):
    checkpoint = {
        'net': net.state_dict(),
        'optimizer': optimizer.state_dict(),
        'train_acc': train_acc,
        'test_acc': test_acc
    }
    torch.save(checkpoint, '/content/drive/My Drive/{}_{}_best.chk'.format(net_name, param))

In [0]:
train(res_net, ddms_train_loader, ddms_criterion, ddms_optimizer_res_net, 'res_net_custom_cropped_exp_2', 100)

End of epoch 1, Loss 0.657499263567083
Accuracy of the network on the images: 76 %
End of epoch 2, Loss 0.583149851770962
Accuracy of the network on the images: 72 %
End of epoch 3, Loss 0.5526510007241193
Accuracy of the network on the images: 79 %
End of epoch 4, Loss 0.5356269629562602
Accuracy of the network on the images: 78 %
End of epoch 5, Loss 0.5392786863972159
Accuracy of the network on the images: 77 %
End of epoch 6, Loss 0.5258482543861165
Accuracy of the network on the images: 78 %
End of epoch 7, Loss 0.5260945961755865
Accuracy of the network on the images: 79 %
End of epoch 8, Loss 0.49786310511476856
Accuracy of the network on the images: 79 %
End of epoch 9, Loss 0.5019566374666551
Accuracy of the network on the images: 79 %
End of epoch 10, Loss 0.4954571559148676
Accuracy of the network on the images: 81 %
End of epoch 11, Loss 0.4826545992318322
Accuracy of the network on the images: 77 %
End of epoch 12, Loss 0.4838136865812189
Accuracy of the network on the ima

In [9]:
# Demonstration that saved checkpoint can be loaded 
ckpt = torch.load('/content/drive/My Drive/res_net_custom_cropped_exp_2_test_best.chk')
res_net_saved = resnet18(pretrained=True)
for param in res_net_saved.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = res_net_saved.fc.in_features
res_net_saved.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=2, bias=True))
res_net_saved.load_state_dict(ckpt['net'])
res_net_saved.cuda()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 78.8MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
# Separate feature generator (everything except FC block)
fgen = nn.Sequential(*list(res_net_saved.children())[:-1])

In [0]:
iter(ddms_test_loader).next()[0].shape

torch.Size([16, 1, 224, 224])

In [0]:
# See how many images are in training set and test set
total = 0
test_total = 0
for x, _ in ddms_train_loader:
    total += x.shape[0]
print(total)
for x, _ in ddms_test_loader:
    test_total += x.shape[0]
print(test_total)

1357
339


In [0]:
# we will generate 10 times more training data than the original training data size using data augmentation
train_features = np.zeros((total * 10, num_ftrs)).astype('float32')
train_labels = np.zeros(total * 10).astype('int64')
test_features = np.zeros((test_total, num_ftrs)).astype('float32')
test_labels = np.zeros(test_total).astype('int64')
index  = 0
for e in range(10):
    for data, labels in ddms_train_loader:
        # each generated train feature will be a 512 element vector
        train_features[index:index + data.shape[0]] = fgen(data.expand(-1, 3, -1, -1)).view(data.shape[0], num_ftrs).cpu()
        # labels is in GPU, so transfer it to CPU first to assign to the numpy array
        train_labels[index:index + data.shape[0]] = labels.cpu()
        index += data.shape[0]

index = 0
for data, labels in ddms_test_loader:
    test_features[index:index + data.shape[0]] = fgen(data.expand(-1, 3, -1, -1)).view(data.shape[0], num_ftrs).cpu()
    test_labels[index:index + data.shape[0]] = labels.cpu()
    index += data.shape[0]


In [0]:
# Network for Bayesian purpose, 3 fully connected layers
class BayesNet(nn.Module):
    def __init__(self, num_ftrs):
        super(BayesNet, self).__init__()
        self.fc1 = nn.Linear(in_features=num_ftrs, out_features=256, bias=True)
        self.fc2 = nn.Linear(in_features=256, out_features=128, bias=True)
        self.fc3 = nn.Linear(in_features=128, out_features=2, bias=True)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [17]:
bnet = BayesNet(num_ftrs)
bnet.cuda()

BayesNet(
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)

In [0]:
class DataSetBayes(Dataset):

    def __init__(self, data, labels):
        """ Intialize the dataset
        """
        self.data = data
        self.labels = labels
                
        self.len = self.data.shape[0]

    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        return self.data[index], self.labels[index]

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [0]:
bayes_train_loader = torch.utils.data.DataLoader(DataSetBayes(train_features, train_labels), 
                                                batch_size=8, shuffle=True)
bayes_test_loader = torch.utils.data.DataLoader(DataSetBayes(test_features, test_labels), 
                                               batch_size=8, shuffle=True)

In [0]:
# functions for Bayesian training needed in Pyro, model() and guide() 
log_softmax = nn.LogSoftmax(dim=1)
def model_bayes_net(x_data, y_data):
    fc1w_prior = Normal(loc=torch.zeros_like(bnet.fc1.weight).cuda(), scale=torch.ones_like(bnet.fc1.weight).cuda())
    fc1b_prior = Normal(loc=torch.zeros_like(bnet.fc1.bias).cuda(), scale=torch.ones_like(bnet.fc1.bias).cuda())
    
    fc2w_prior = Normal(loc=torch.zeros_like(bnet.fc2.weight).cuda(), scale=torch.ones_like(bnet.fc2.weight).cuda())
    fc2b_prior = Normal(loc=torch.zeros_like(bnet.fc2.bias).cuda(), scale=torch.ones_like(bnet.fc2.bias).cuda())
    
    fc3w_prior = Normal(loc=torch.zeros_like(bnet.fc3.weight).cuda(), scale=torch.ones_like(bnet.fc3.weight).cuda())
    fc3b_prior = Normal(loc=torch.zeros_like(bnet.fc3.bias).cuda(), scale=torch.ones_like(bnet.fc3.bias).cuda())

    
    priors = {'fc1.weight': fc1w_prior, 
              'fc1.bias': fc1b_prior, 
              'fc2.weight': fc2w_prior, 
              'fc2.bias': fc2b_prior, 
              'fc3.weight': fc3w_prior, 
              'fc3.bias': fc3b_prior}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", bnet, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()

    lhat = log_softmax(lifted_reg_model(x_data))
    
    pyro.sample("obs", Categorical(logits=lhat), obs=y_data)

In [0]:
softplus = torch.nn.Softplus()

def guide_bayes_net(x_data, y_data):
    # First fully connected layer weight distribution priors
    fc1w_mu = torch.randn_like(bnet.fc1.weight).cuda()
    fc1w_sigma = torch.randn_like(bnet.fc1.weight).cuda()
    fc1w_mu_param = pyro.param("fc1w_mu", fc1w_mu)
    fc1w_sigma_param = softplus(pyro.param("fc1w_sigma", fc1w_sigma))
    fc1w_prior = Normal(loc=fc1w_mu_param, scale=fc1w_sigma_param)
    # First fully connected layer bias distribution priors
    fc1b_mu = torch.randn_like(bnet.fc1.bias).cuda()
    fc1b_sigma = torch.randn_like(bnet.fc1.bias).cuda()
    fc1b_mu_param = pyro.param("fc1b_mu", fc1b_mu)
    fc1b_sigma_param = softplus(pyro.param("fc1b_sigma", fc1b_sigma))
    fc1b_prior = Normal(loc=fc1b_mu_param, scale=fc1b_sigma_param)
    # Second fully connected layer weight distribution priors
    fc2w_mu = torch.randn_like(bnet.fc2.weight).cuda()
    fc2w_sigma = torch.randn_like(bnet.fc2.weight).cuda()
    fc2w_mu_param = pyro.param("fc2w_mu", fc2w_mu)
    fc2w_sigma_param = softplus(pyro.param("fc2w_sigma", fc2w_sigma))
    fc2w_prior = Normal(loc=fc2w_mu_param, scale=fc2w_sigma_param)
    # Second fully connected layer bias distribution priors
    fc2b_mu = torch.randn_like(bnet.fc2.bias).cuda()
    fc2b_sigma = torch.randn_like(bnet.fc2.bias).cuda()
    fc2b_mu_param = pyro.param("fc2b_mu", fc2b_mu)
    fc2b_sigma_param = softplus(pyro.param("fc2b_sigma", fc2b_sigma))
    fc2b_prior = Normal(loc=fc2b_mu_param, scale=fc2b_sigma_param)
    # Third fully connected layer weight distribution priors
    fc3w_mu = torch.randn_like(bnet.fc3.weight).cuda()
    fc3w_sigma = torch.randn_like(bnet.fc3.weight).cuda()
    fc3w_mu_param = pyro.param("fc3w_mu", fc3w_mu)
    fc3w_sigma_param = softplus(pyro.param("fc3w_sigma", fc3w_sigma))
    fc3w_prior = Normal(loc=fc3w_mu_param, scale=fc3w_sigma_param)
    # Third fully connected layer bias distribution priors
    fc3b_mu = torch.randn_like(bnet.fc3.bias).cuda()
    fc3b_sigma = torch.randn_like(bnet.fc3.bias).cuda()
    fc3b_mu_param = pyro.param("fc3b_mu", fc3b_mu)
    fc3b_sigma_param = softplus(pyro.param("fc3b_sigma", fc3b_sigma))
    fc3b_prior = Normal(loc=fc3b_mu_param, scale=fc3b_sigma_param)
    
    priors = {'fc1.weight': fc1w_prior, 
              'fc1.bias': fc1b_prior, 
              'fc2.weight': fc2w_prior, 
              'fc2.bias': fc2b_prior, 
              'fc3.weight': fc3w_prior, 
              'fc3.bias': fc3b_prior}
    
    lifted_module = pyro.random_module("module", bnet, priors)
    
    return lifted_module()

In [0]:
# function for different learning rates at different layers
def get_optim_args(module_name, param_name):
    if param_name == 'fc1.weight' or param_name == 'fc1.bias':
        return {"lr": 0.0001, 'betas': (0.5, 0.999)}
    elif param_name == 'fc2.weight' or param_name == 'fc2.bias':
        return {"lr": 0.001, 'betas': (0.5, 0.999)}
    else:
        return {"lr": 0.002, 'betas': (0.5, 0.999)}
    
optim = torch.optim.Adam
scheduler = pyro.optim.StepLR({'optimizer': optim,  'optim_args': get_optim_args,
                               'step_size': 5, 'gamma': 0.1})

# define stochastic variational inference
svi = SVI(model_bayes_net, guide_bayes_net, scheduler, loss=Trace_ELBO())

In [0]:
# bayesian training

num_iterations = 10
loss = 0

for j in range(num_iterations):
    loss = 0
    for batch_id, data in enumerate(bayes_train_loader):
        # calculate the loss and take a gradient step for updating the weight distributions
        loss += svi.step(data[0].cuda(), data[1].cuda())
    normalizer_train = len(bayes_train_loader.dataset)
    total_epoch_loss_train = loss / normalizer_train
    scheduler.step(epoch=j)
    
    print("Epoch ", j, " Loss ", total_epoch_loss_train)

Epoch  0  Loss  15638.837418734029
Epoch  1  Loss  8528.736376881132
Epoch  2  Loss  3899.6411283448256
Epoch  3  Loss  1551.1354187588793
Epoch  4  Loss  622.9060791623598
Epoch  5  Loss  313.27244874770446
Epoch  6  Loss  240.09437875786196
Epoch  7  Loss  221.8147793499586
Epoch  8  Loss  209.39330723049747
Epoch  9  Loss  199.755452349664


In [0]:
# evaluation with the sampled networks

num_samples = 1000
def predict(x):
    sampled_models = [guide_bayes_net(None, None) for _ in range(num_samples)]
    yhats = [model(x).data for model in sampled_models]
    mean = torch.mean(torch.stack(yhats), 0)
    return np.argmax(mean.cpu().numpy(), axis=1)

# forced prediction based on sum of probability for each label across all 1000 networks
print('Prediction when network is forced to predict')
correct = 0
total = 0
for j, data in enumerate(bayes_test_loader):
    dat, labels = data
    predicted = predict(dat.cuda())
    total += labels.size(0)
    correct += (predicted == labels.numpy()).sum().item()
print("accuracy: %d %%" % (100 * correct / total))

Prediction when network is forced to predict
accuracy: 81 %


In [0]:
classes = ('BENIGN', 'MALIGNANT')

In [0]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    
    fig, ax = plt.subplots(figsize=(1, 1))
    ax.imshow(npimg,  cmap='gray', interpolation='nearest')
    plt.show()

In [0]:
num_samples = 1000
sampled_models = [guide_bayes_net(None, None) for _ in range(num_samples)]

In [0]:
# helper method for getting prediction probabilities for each sampled network
def give_uncertainities(x):
    yhats = [F.log_softmax(model(x).data, 1).detach().cpu().numpy() for model in sampled_models]
    return np.asarray(yhats)

In [0]:
# method for tuning N and P and calculating accuracy
# N = threshold
# P = min_prob
def test_batch(images, labels, threshold=0.8, min_prob=0.7, plot=True):
    y = give_uncertainities(images)
    predicted_for_images = 0
    correct_predictions=0

    for i in range(len(labels)):
    
        if(plot):
            print("Real: ",labels[i].item())
            fig, axs = plt.subplots(1, 3, sharey=True,figsize=(20,2))
    
        all_labels_prob = []
    
        highted_something = False
    
        for j in range(len(classes)):
        
            highlight=False
        
            histo = []
            histo_exp = []
        
            for z in range(y.shape[0]):
                histo.append(y[z][i][j])
                histo_exp.append(np.exp(y[z][i][j]))
            
            histo_exp = np.array(histo_exp)
            count = np.where(histo_exp > min_prob)[0].shape[0]
  
            if count > num_samples * threshold:
              highlight = True
        
            all_labels_prob.append(count)
            
            if(plot):
            
                N, bins, patches = axs[j].hist(histo, bins=8, color = "lightgray", lw=0,  weights=np.ones(len(histo)) / len(histo), density=False)
                axs[j].set_title(str(j)+" ("+str(round(prob,2))+")") 
        
            if(highlight):
            
                highted_something = True
                
                if(plot):

                    # We'll color code by height, but you could use any scalar
                    fracs = N / N.max()

                    # we need to normalize the data to 0..1 for the full range of the colormap
                    norm = colors.Normalize(fracs.min(), fracs.max())

                    # Now, we'll loop through our objects and set the color of each accordingly
                    for thisfrac, thispatch in zip(fracs, patches):
                        color = plt.cm.viridis(norm(thisfrac))
                        thispatch.set_facecolor(color)

    
        if(plot):
            plt.show()
    
        predicted = np.argmax(all_labels_prob)
    
        if(highted_something):
            predicted_for_images+=1
            if(labels[i].item()==predicted):
                if(plot):
                    print("Correct")
                correct_predictions +=1.0
            else:
                if(plot):
                    print("Incorrect :()")
        else:
            if(plot):
                print("Undecided.")
        
        if(plot):
            imshow(images[i].squeeze())
        
    
    if(plot):
        print("Summary")
        print("Total images: ",len(labels))
        print("Predicted for: ",predicted_for_images)
        print("Accuracy when predicted: ",correct_predictions/predicted_for_images)
        
    return len(labels), correct_predictions, predicted_for_images

In [0]:
# Prediction when network can decide not to predict
print('Prediction when network can refuse')
correct = 0
total = 0
total_predicted_for = 0
for j, data in enumerate(bayes_test_loader):
    images, labels = data
    
    total_minibatch, correct_minibatch, predictions_minibatch = test_batch(images.cuda(), labels.cuda(), 0.75, 0.7, plot=False)
    total += total_minibatch
    correct += correct_minibatch
    total_predicted_for += predictions_minibatch

print("Total images: ", total)
print("Skipped: ", total-total_predicted_for)
print("Accuracy when made predictions: %d %%" % (100 * correct / total_predicted_for))

Prediction when network can refuse
Total images:  339
Skipped:  337
Accuracy when made predictions: 100 %


In [2]:
print('N and P tuning result generation')
results = []
ns = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2]
ps = [0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2]

for N in ns:
    for P in ps:
        result = []
        result.append(N)
        result.append(P)
        correct = 0
        total = 0
        total_predicted_for = 0
        for j, data in enumerate(bayes_test_loader):
            images, labels = data

            total_minibatch, correct_minibatch, predictions_minibatch = test_batch(images.cuda(), labels.cuda(), N, P, plot=False)
            total += total_minibatch
            correct += correct_minibatch
            total_predicted_for += predictions_minibatch
        result.append(total)
        result.append(total-total_predicted_for)
        if total_predicted_for > 0:
            result.append(correct / total_predicted_for)
        else:
            result.append(-1)
        results.append(result)

N and P tuning result generation


In [0]:
results_df = pd.DataFrame(results, columns=["N", "P", "Total Images", "Skipped Images", "Accuracy"])
results_df['Accuracy']
results_df.loc[results_df['Accuracy'] == -1, 'Accuracy'] = np.NaN
results_df['Coverage'] = (results_df['Total Images'] - results_df['Skipped Images']) / results_df['Total Images']
results_df.to_csv('/content/drive/My Drive/tuning_results_3.csv')
results_df

,N,P,Total Images,Skipped Images,Accuracy,Coverage
0,0.95,0.95,339,339,NaN,0.000000
1,0.95,0.90,339,339,NaN,0.000000
2,0.95,0.85,339,339,NaN,0.000000
3,0.95,0.80,339,339,NaN,0.000000
4,0.95,0.75,339,339,NaN,0.000000
5,0.95,0.70,339,339,NaN,0.000000
6,0.95,0.65,339,339,NaN,0.000000
7,0.95,0.60,339,339,NaN,0.000000
8,0.95,0.55,339,339,NaN,0.000000
9,0.95,0.50,339,339,NaN,0.000000


In [ ]:
results_df[results_df['N'] == 0.6]

In [0]:
results_df.to_csv('/content/drive/My Drive/tuning_results_3_ext.csv')

In [0]:
# save the training labels to a pickle file for later use
with open ('/content/drive/My Drive/train_labels.pkl', 'wb') as f:
    pickle.dump(train_labels, f)


In [0]:
# save the 1000 sampled models to a checkpoint file for later use
checkpoints = {}
for i, model in enumerate(sampled_models):
    checkpoints['model{}'.format(i)] = model.state_dict()
torch.save(checkpoints, '/content/drive/My Drive/bayesian_models.chk')

In [0]:
# load the above saved checkpoint
ckptb = torch.load('/content/drive/My Drive/bayesian_models.chk')

In [0]:
# verify that the weights are there
bnet2 = BayesNet(num_ftrs)
bnet2.load_state_dict(ckptb['model10'])
F.log_softmax(bnet2(torch.tensor(test_features)), 1)

tensor([[-1.4479e+02,  0.0000e+00],
        [ 0.0000e+00, -4.7416e+02],
        [-7.9243e+02,  0.0000e+00],
        [ 0.0000e+00, -9.1107e+02],
        [-8.2344e+02,  0.0000e+00],
        [-1.4966e+00, -2.5346e-01],
        [ 0.0000e+00, -1.3178e+02],
        [-1.4924e+00, -2.5468e-01],
        [ 0.0000e+00, -2.1102e+02],
        [-4.6472e+01,  0.0000e+00],
        [-1.4966e+00, -2.5346e-01],
        [-1.0362e+03,  0.0000e+00],
        [ 0.0000e+00, -1.7131e+02],
        [-1.1197e+02,  0.0000e+00],
        [-2.0199e+02,  0.0000e+00],
        [-4.9591e-05, -9.9187e+00],
        [-5.4628e-01, -8.6536e-01],
        [-1.4966e+00, -2.5346e-01],
        [ 0.0000e+00, -2.1522e+02],
        [ 0.0000e+00, -3.4003e+02],
        [ 0.0000e+00, -2.0656e+02],
        [-7.1526e-07, -1.4197e+01],
        [ 0.0000e+00, -4.3608e+02],
        [-5.6962e+02,  0.0000e+00],
        [ 0.0000e+00, -2.8464e+02],
        [ 0.0000e+00, -1.6834e+02],
        [-7.9937e+00, -3.3760e-04],
        [-7.4389e+01,  0.000